In [34]:


import os
import pickle
import zlib
import numpy as np
from netCDF4 import Dataset, date2num, num2date
import datetime
import joblib
import cftime

file_path = '/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/spin21.pkz'
pkz_folder = '/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/'
output_folder = '/home/bianca/bianca/CAETE-DVM-JP/CAETE-DVM/outputs/CAX-ISIMIP/gridcell183-257/output_folder'

with open(file_path, 'rb') as fh:
   dt = joblib.load(fh)
   
   print(dt.keys()) # list the available keys for the ouputs
   # 'calendar', 'time_unit', 'sind', 'eind'
   print(dt['calendar'])
   print(dt['time_unit'])
   print(dt['sind'])
   print(dt['eind'])

def create_lband(res=0.5):
    #creates an array of longitudes based on a spacing of res. that is 0.5º
    lon = np.arange(-179.75, 180, res, dtype=np.float64)[201:272]
    lat = np.arange(89.75, -90, -res, dtype=np.float64)[160:221][::-1]

    #defines the lower and upper bounds of each latitude band. 
    half = res / 2.0

    #the following vars refer to the latitude and longitude 
    # intervals associated with each specific point in a geographic grid
    latbnd = np.array([[l - half, l + half] for l in lat])
    lonbnd = np.array([[l - half, l + half] for l in lon])
    
    return lat, latbnd, lon, lonbnd

def set_historical_stime(new_descr=True):

    global TIME_UNITS, CALENDAR
    TIME_UNITS = "days since 1860-01-01 00:00:00"
    CALENDAR = "proleptic_gregorian"

def get_var_metadata(var):

    vunits = {'header': ['long_name', 'unit', 'standart_name'],

              'rsds': ['short_wav_rad_down', 'W m-2', 'rsds'],
              'wind': ['wind_velocity', 'm s-1', 'wind'],
              'ps': ['sur_pressure', 'Pa', 'ps'],
              'tas': ['sur_temperature_2m', 'celcius', 'tas'],
              'tsoil': ['soil_temperature', 'celcius', 'soil_temp'],
              'pr': ['precipitation', 'Kg m-2 month-1', 'pr'],
              'litter_l': ['Litter C flux - leaf', 'g m-2 day-1', 'll'],
              'cwd': ['Litter C flux - wood', 'g m-2 day-1', 'cwd'],
              'litter_fr': ['Litter C flux fine root', 'g m-2 day-1', 'lr'],
              'litter_n': ['Litter Nitrogen Flux', 'g m-2 day-1', 'ln'],
              'litter_p': ['Litter phosphorus flux', 'g m-2 day-1', 'lp'],
              'sto_c': ['PLant Reserve Carbon', 'g m-2', 'sto_c'],
              'sto_n': ['Pant Reserve Nitrogen', 'g m-2', 'sto_n'],
              'sto_p': ['Plant Reserve Phosphorus', 'g m-2', 'sto_p'],
              'c_cost': ['Carbon costs of Nutrients Uptake', 'g m-2 day-1', 'cc'],
              'wsoil': ['soil_water_content-wsoil', 'kg m-2', 'mrso'],
              'evapm': ['evapotranspiration', 'kg m-2 day-1', 'et'],
              'emaxm': ['potent. evapotrasnpiration', 'kg m-2 day-1', 'etpot'],
              'runom': ['total_runoff', 'kg m-2 day-1', 'mrro'],
              'aresp': ['autothrophic respiration', 'kg m-2 year-1', 'ar'],
              'photo': ['gross primary productivity', 'kg m-2 year-1', 'gpp'],
              'npp': ['net primary productivity', 'kg m-2 year-1', 'npp'],
              'lai': ['Leaf Area Index - LAI - BIG LEAF', 'm2 m-2', 'lai'],
              'rcm': ['stomatal resistence', 's m-1', 'rcm'],
              'hresp': ['Soil heterotrophic respiration', 'g m-2 day-1', 'hr'],
              'nupt': ['Nitrogen uptake', 'g m-2 day-1', 'nupt'],
              'pupt': ['Phosphorus uptake', 'g m-2 day-1', 'pupt'],
              'csoil': ['Soil Organic Carbon', 'g m-2', 'csoil'],
              'org_n': ['Soil Organic Nitrogen', 'g m-2', 'org_n'],
              'org_p': ['Soil Organic Phosphorus', 'g m-2', 'org_p'],
              'inorg_n': ['Soil Inorganic Nitrogen', 'g m-2', 'inorg_n'],
              'inorg_p': ['Soil Inorganic Phosphorus', 'g m-2', 'inorg_p'],
              'sorbed_p': ['Soil Sorbed Phosphorus', 'g m-2', 'sorbed_p'],
              'nmin': ['Soil Inorganic Nitrogen (solution)', 'g m-2', 'nmin'],
              'pmin': ['Soil Inorganic Phosphorus (solution)', 'g m-2', 'pmin'],
              'rm': ['maintenance respiration', 'kg m-2 year-1', 'rm'],
              'rg': ['growth respiration', 'kg m-2 year-1', 'rg'],
              'wue': ['water use efficiency', '1', 'wue'],
              'vcmax': ['maximum RuBisCo activity', 'mol m-2 s-1', 'vcmax'],
              'sla': ['specfic leaf area', 'm2 g-1', 'sla'],
              'cue': ['Carbon use efficiency', '1', 'cue'],
              'cawood': ['C in woody tissues', 'kg m-2', 'cawood'],
              'cfroot': ['C in fine roots', 'kg m-2', 'cfroot'],
              'cleaf': ['C in leaves', 'kg m-2', 'cleaf'],
              'cmass': ['total Carbon -Biomass', 'kg m-2', 'cmass'],
              'leaf_nolim': ['no lim. in leaf growth', 'Time fraction', 'leaf_nolim'],
              'leaf_lim_n': ['N lim. growth L', 'Time fraction', 'leaf_lim_n'],
              'leaf_lim_p': ['P lim. growth L', 'Time fraction', 'leaf_lim_p'],
              'leaf_colim_n': ['colim N L', 'Time fraction', 'leaf_colim_n'],
              'leaf_colim_p': ['colim P L', 'Time fraction', 'leaf_colim_p'],
              'leaf_colim_np': ['colim NP L', 'Time fraction', 'leaf_colim_np'],
              'wood_nolim': ['no lim. in wood growth', 'Time fraction', 'wood_nolim'],
              'wood_lim_n': ['N lim. growth W', 'Time fraction', 'wood_lim_n'],
              'wood_lim_p': ['P lim. growth W', 'Time fraction', 'wood_lim_p'],
              'wood_colim_n': ['colim N W', 'Time fraction', 'wood_colim_n'],
              'wood_colim_p': ['colim P W', 'Time fraction', 'wood_colim_p'],
              'wood_colim_np': ['colim NP W', 'Time fraction', 'wood_colim_np'],
              'root_nolim': ['no lim. in root growth', 'Time fraction', 'root_nolim'],
              'root_lim_n': ['N lim. growth R', 'Time fraction', 'root_lim_n'],
              'root_lim_p': ['P lim. growth R', 'Time fraction', 'root_lim_p'],
              'root_colim_n': ['colim N R', 'Time fraction', 'root_colim_n'],
              'root_colim_p': ['colim P R', 'Time fraction', 'root_colim_p'],
              'root_colim_np': ['colim NP R', 'Time fraction', 'root_colim_np'],
              'upt_stratN0': ['passive N uptake', 'Time fraction', 'upt_stratN0'],
              'upt_stratN1': ['active N uptake via AM ROOT surface - nma', 'Time fraction', 'upt_stratN1'],
              'upt_stratN2': ['active N uptake via ECM ROOT surface - nme', 'Time fraction', 'upt_stratN2'],
              'upt_stratN3': ['active N uptake via AM surface - am', 'Time fraction', 'upt_stratN3'],
              'upt_stratN4': ['active N uptake via ECM surface - em', 'Time fraction', 'upt_stratN4'],
              'upt_stratN6': ['N uptake via ECM NITROGENASE activity - em0', 'Time fraction', 'upt_stratN6'],
              'upt_stratP0': ['passive P uptake', 'Time fraction', 'upt_stratP0'],
              'upt_stratP1': ['active P uptake via AM ROOT surface - nma', 'Time fraction', 'upt_stratP1'],
              'upt_stratP2': ['active P uptake via ECM ROOT surface - nme', 'Time fraction', 'upt_stratP2'],
              'upt_stratP3': ['active P uptake via AM surface - am', 'Time fraction', 'upt_stratP3'],
              'upt_stratP4': ['active P uptake via ECMM surface - em', 'Time fraction', 'upt_stratP4'],
              'upt_stratP5': ['P uptake via ROOT AM Ptase activity - ram-ap', 'Time fraction', 'upt_stratP5'],
              'upt_stratP6': ['P uptake via ROOT ECM Ptase activity - rem-ap', 'Time fraction', 'upt_stratP6'],
              'upt_stratP7': ['P uptake via AM Ptase activity - amap', 'Time fraction', 'upt_stratP7'],
              'upt_stratP8': ['P uptake via exudates activity - em0x', 'Time fraction', 'upt_stratP8'],
              'g1': ['G1 param - Stomatal Resistence model', 'hPA', 'g1'],
              'resopfrac': ['Leaf resorpton fraction N & P', '%', 'resopfrac'],
              'tleaf': ['leaf C residence time', 'years', 'tleaf'],
              'twood': ['wood C residence time', 'years', 'twood'],
              'troot': ['fine root C residence time', 'years', 'troot'],
              'aleaf': ['allocation coefficients for leaf', '1', 'aleaf'],
              'awood': ['allocation coefficients for wood', '1', 'awood'],
              'aroot': ['allocation coefficients for root', '1', 'aroot'],
              'c4': ['c4 photosynthesis pathway', '1', 'c4'],
              'leaf_n2c': ['leaf N:C', 'g g-1', 'leaf_n2c'],
              'awood_n2c': ['wood tissues N:C', 'g g-1', 'awood_n2c'],
              'froot_n2c': ['fine root N:C', 'g g-1', 'froot_n2c'],
              'leaf_p2c': ['leaf P:C', 'g g-1', 'leaf_p2c'],
              'awood_p2c': ['wood tissues P:C', 'g g-1', 'awood_p2c'],
              'froot_p2c': ['fine root P:C', 'g g-1', 'froot_p2c'],
              'amp': ['Percentage of fine root colonized by AM', '%', 'amp'],
              'pdia': ['NPP alocated to N fixers', 'fraction_of_npp', 'pdia'],
              'ls': ['Living Plant Life Strategies', '1', 'ls']}

    out = {}
    for v in var:
        out[v] = vunits[v]
    return out

# Função para converter dados de PKZ para NetCDF
def convert_pkz_to_netcdf(pkz_folder, output_folder):
    
    
    # Certifique-se de que a pasta de saída exista
    os.makedirs(output_folder, exist_ok=True)

    set_historical_stime()

    NO_DATA = [-9999.0, -9999.0]
    time_units = TIME_UNITS
    calendar = CALENDAR
   
    #prepare lat/lon
    geo_v = create_lband()
    lat = geo_v[0]
    lat_bnds = geo_v[1]
    lon = geo_v[2]
    lon_bnds = geo_v[3]

    # Variáveis (from pkz file)
    variables = ['emaxm', 'tsoil', 'photo', 'aresp', 'npp', 'cleaf', 'cawood', 'cfroot']
    npp = variables[4] 

    # Resolução da grade
    resolucao = 0.5

    # Calcula o número total de pontos de grade
    num_latitudes = len(lat)
    num_longitudes = len(lon)

    # Certifique-se de que a pasta de saída exista
    os.makedirs(output_folder, exist_ok=True)
   

    var_attrs = get_var_metadata(variables)

    for i, v in enumerate(variables):
        print(i,v)

    # # Loop através das variáveis
    # for variable in variables:
    #     # Inicializa o NetCDF
    #     nc_filename = os.path.join(output_folder, f"{variable}.nc")
    #     with Dataset(nc_filename, 'w', format='NETCDF4') as nc:

    #         # Dimensões e variáveis
    #         nc.createDimension('time', None)
    #         nc.createDimension('lat', num_latitudes)
    #         nc.createDimension('lon', num_longitudes)

    #         time_var = nc.createVariable('time', 'f8', ('time',))
    #         lat_var = nc.createVariable('lat', 'f4', ('lat',))
    #         lon_var = nc.createVariable('lon', 'f4', ('lon',))
    #           # Escolhe um nome para a variável NetCDF
    #         nc_var_name = f"{variable}_data"
    #         data_var = nc.createVariable(nc_var_name, 'f4', ('time', 'lat', 'lon'), fill_value=NO_DATA)

    #         # Adiciona atributos
    #         time_var.units = time_units
    #         time_var.calendar = calendar
    #         lat_var.units = 'degrees_north'
    #         lon_var.units = 'degrees_east'
    #         data_var.units = npp


convert_pkz_to_netcdf(pkz_folder, output_folder)

dict_keys(['emaxm', 'tsoil', 'photo', 'aresp', 'npp', 'lai', 'csoil', 'inorg_n', 'inorg_p', 'sorbed_n', 'sorbed_p', 'snc', 'hresp', 'rcm', 'f5', 'runom', 'evapm', 'wsoil', 'swsoil', 'rm', 'rg', 'cleaf', 'cawood', 'cfroot', 'area', 'wue', 'cue', 'cdef', 'nmin', 'pmin', 'vcmax', 'specific_la', 'nupt', 'pupt', 'litter_l', 'cwd', 'litter_fr', 'lnc', 'ls', 'lim_status', 'c_cost', 'u_strat', 'storage_pool', 'calendar', 'time_unit', 'sind', 'eind'])
proleptic_gregorian
days since 1860-01-01 00:00:00
43464
57343
0 emaxm
1 tsoil
2 photo
3 aresp
4 npp
5 cleaf
6 cawood
7 cfroot
